# Mapping back short reads to assembly

In [4]:
R1=/input/genomic/plant/Vaccinium/myrtillus/Genome/Illumina/AGRF_CAGRF20139_HMHTHDSXX/Clean_Billbery_HMHTHDSXX_CTAGCGCT-GTGTAGAC_L001_R1.fastq.gz
R2=/input/genomic/plant/Vaccinium/myrtillus/Genome/Illumina/AGRF_CAGRF20139_HMHTHDSXX/Clean_Billbery_HMHTHDSXX_CTAGCGCT-GTGTAGAC_L001_R2.fastq.gz

In [5]:
GENOME=/output/genomic/plant/Vaccinium/myrtillus/Genome/Assembly/bilberry_chr01-12_plusUnanchoredSeqs.NCBI.fasta
CHRS=022.JuiceBox_visualisation/BilberryGenome_chrs.fasta

In [6]:
module load bwa
module load samtools
module list

Currently Loaded Modulefiles:
  1) powerPlant/core    7) asub/2.2          13) vcflib/73b7f13
  2) texlive/20151117   8) unixODBC/2.3.0    14) freebayes/1.1.0
  3) pandoc/1.19.2      9) JAGS/4.2.0        15) bwa/0.7.17
  4) git/2.21.0        10) gdal/2.4.0        16) samtools/1.9
  5) perlbrew/0.76     11) proj/5.2.0
  6) perl/5.28.0       12) R/3.3.0


In [6]:
mkdir 020.mappingBackRateCheck

In [7]:
WDR=020.mappingBackRateCheck

In [14]:
bsub -J bwa_index \
-o $WDR/bwa_index.out \
-e $WOR/bwa_index.err \
"bwa index $CHRS"

Job <887272> is submitted to default queue <lowpriority>.


In [8]:
WDR=020.mappingBackRateCheck

In [15]:
bsub -J bwa -n 40 -o $WDR/bwa_chrs.out -e $WDR/bwa_chrs.err \
"bwa mem -t 40 $CHRS $R1 $R2 \
| samtools view -@ 40 -Sb -o $WDR/mapped_bwa.bam"

Job <887273> is submitted to default queue <lowpriority>.


In [4]:
bsub -J sort -n 10 -o $WDR/samtools_sort_chrs.out -e $WDR/samtools_sort_chrs.err \
"samtools sort -@ 10 -o $WDR/mapped_bwa.sorted.bam $WDR/mapped_bwa.bam"

Job <887664> is submitted to default queue <lowpriority>.


In [12]:
bsub -J index -o $WDR/samtools_index_chrs.out -e $WDR/samtools_index_chrs.err \
"samtools index $WDR/mapped_bwa.sorted.bam"

Job <887701> is submitted to default queue <lowpriority>.


## freebayes calling

In [17]:
samtools faidx $CHRS

In [1]:
module load freebayes/1.1.0
module list

Currently Loaded Modulefiles:
  1) powerPlant/core    6) perl/5.28.0       11) proj/5.2.0
  2) texlive/20151117   7) asub/2.2          12) R/3.3.0
  3) pandoc/1.19.2      8) unixODBC/2.3.0    13) vcflib/73b7f13
  4) git/2.21.0         9) JAGS/4.2.0        14) freebayes/1.1.0
  5) perlbrew/0.76     10) gdal/2.4.0


In [2]:
FREEBAYES=/software/bioinformatics/freebayes-1.1.0/scripts

In [3]:
export TMPDIR=/workspace/hraczw/github/hoki_genomics/007.mappingAndCalling

In [9]:
bsub << EOF
#!/bin/bash
#BSUB -J freebayes
#BSUB -o $WDR/freebayes.out 
#BSUB -e $WDR/freebayes.err
#BSUB -R "span[hosts=1]"
#BSUB -m wkoppg34
#BSUB -n 40
#BSUB -q priority
#BSUB -P P/536002/01

$FREEBAYES/freebayes-parallel <($FREEBAYES/fasta_generate_regions.py $CHRS.fai 100000) 80 \
-f $CHRS $WDR/mapped_bwa.sorted.bam -p 4 -C 3 -m 10 \
--min-coverage 15 --max-coverage 500 >$WDR/freebayes_output.vcf

EOF


Job <912948> is submitted to queue <priority>.
